In [74]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from load_data import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

def compute_loss(y, tx, w, l=0):
    # l = 0 for MSE 
    """Calculate the loss using either MSE or MAE.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2,). The vector of model parameters.

    Returns:
        the value of the loss (a scalar), corresponding to the input parameters w.
    """

    if l == 0:
      # ***************************************************
      # INSERT YOUR CODE HERE
      #compute loss by MSE
      e =  np.dot(tx, w)
      e = y - e
      N = y.shape[0]
      L = 1/(2*N) * e.T * e

      return L.sum()
      # ***************************************************
    else:
      # ***************************************************
      # INSERT YOUR CODE HERE
      #compute loss by MAE
      e =  np.dot(tx, w)
      e = y - e
      N = y.shape[0]
      L = 1/(2*N) * e

      return L.sum()

In [3]:
def least_squares(y, tx):
    """Calculate the least squares solution.
       returns mse, and optimal weights.
    
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
    
    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
        mse: scalar.

    >>> least_squares(np.array([0.1,0.2]), np.array([[2.3, 3.2], [1., 0.1]]))
    (array([ 0.21212121, -0.12121212]), 8.666684749742561e-33)
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    w = np.linalg.solve(tx.T.dot(tx), tx.T.dot(y))
    mse = compute_loss(y, tx, w, 0)
    return w, mse
    # returns mse, and optimal weights
    # ***************************************************

In [14]:
def compute_gradient(y, tx, w):
    """Computes the gradient at w.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.
        
    Returns:
        An numpy array of shape (2, ) (same shape as w), containing the gradient of the loss at w.
    """
    N = y.shape[0]
    e = y -np.dot(tx, w)
    G = -1/N * tx.T.dot(e)
    return G, e

#gradient test
# w = np.zeros(2,)
# #w = np.array([50, 10])
# w = np.array([100, 20])
# g,e = compute_gradient(y, tx, w)
# print(g.shape)
# print(g)

In [55]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of GD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of GD 
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    best_loss = 10e50
    best_w = initial_w
    for n_iter in range(max_iters):
        g, e = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        losses.append(loss)
        best_w = w if min(losses, default= 10e50) < best_loss else best_w
        best_loss = min(losses) 
        w = w - gamma*g
        ws.append(w)
        print("GD iter. {bi}/{ti}: loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss))

    return best_loss, best_w

In [84]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient at w from just few examples n and their corresponding y_n labels.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.
        
    Returns:
        A numpy array of shape (2, ) (same shape as w), containing the stochastic gradient of the loss at w.
    """
    N = y.shape[0]
    e = y -np.dot(tx, w)
    G = -tx.T.dot(e)/N
    return G, e

def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    """The Stochastic Gradient Descent algorithm (SGD).
            
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic gradient
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SGD 
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    best_w = initial_w
    best_loss = 10e50
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: implement stochastic gradient descent.
        for yi, txi in batch_iter(y, tx, batch_size, num_batches=1):
            g, e = compute_stoch_gradient(yi, txi, w)
            loss = compute_loss(yi, txi, w)
            losses.append(loss)
            best_w = w if min(losses, default= 10e50) < best_loss else best_w
            best_loss = min(losses)
            w = w - gamma*g
            ws.append(w)
        # ***************************************************
        #print("SGD iter. {bi}/{ti}: loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss))
    return best_loss, best_w

In [80]:
ratio = 0.8  #ratio is the percentage of the data allocated for training 
seed = 1  #random seed for data shuffling
x_tr, y_tr, x_te, y_te = preprocess_data() #preprocess intput data from the training and test sets
x_tr, x_v, y_tr, y_v = split_data(x_tr, y_tr, ratio, seed) #split training data into training and validation sets

In [81]:
def generate_w(input_shape):
    w = np.zeros(input_shape[1])
    return w

In [88]:
# Define the parameters of the algorithm.
max_iters = 70
gamma = 0.09
# Initialization
initial_w = generate_w(x_tr.shape)
loss, w = gradient_descent(y_tr, x_tr, initial_w, max_iters, gamma)

In [ ]:
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.08
batch_size = 30
# Initialization
initial_w = generate_w(x_tr.shape)
loss, w = stochastic_gradient_descent(y_tr, x_tr, initial_w, batch_size, max_iters, gamma)

In [89]:
y = x_tr.dot(w)
print(y)

[103.5473965  103.23814279 103.48832684 ... 106.19237983  99.63770114
 101.4176577 ]


In [90]:
print(y_tr)

[115  98  98 ... 115  98  98]
